## The objective of this project was to identify debt-free stocks that have performed well consistently over the past 10 years. 

To identify such stocks, the following criteria was used:

•	Company has never made a loss in the past 10 years -> Positive Revenue and Net Income

•	Lower Debt over the past 10 years -> Debt/Equity Ratio less than 0.5

•	Consistent Performance in terms of Gross Profit Margin and Operating Profit Margin -> Coefficient of Variation, i.e., relative dispersion of data points around the mean to be less than 0.25


## Stages:
>Data Gathering


A list of securities (Equity.csv) was first downloaded from bseindia.com. This list contains stock names and security codes. Morningstar was used to download financial statements – Key Ratios, Balance Sheet, Cash Flow Statement, Profit and Loss Statement


The URL contains either the security code of the stock, or a unique code which could be gathered through a google search therefore it was possible to automate the process using security codes from Equity.csv. 

To trigger the download, a javascript function had to be executed. For this, selenium library was used along with Firefox

>Data Cleaning + Summarizing


The entire statement of Key Ratios was imported, cleaned and summarized in a single row. Process was automated using Security Codes.

>Filtering + Further Extraction of Data


Based on the newly created dataframe, it was possible to filter out stocks which would satisfy our criteria.


## Data Gathering

List of stocks had to be manually downloaded from https://www.bseindia.com/corporates/List_Scrips.html 

In [1]:
#Importing Libraries

import pandas as pd
import os
import selenium.webdriver as webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
import time

In [ ]:
#Configuring Firefox

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'
firefox_driver = os.path.join(os.getcwd(), 'Drivers', 'geckodriver.exe')
firefox_service = Service(firefox_driver)
firefox_options = Options()
firefox_options.set_preference('general.useragent.override', user_agent)
driver = webdriver.Firefox(service = firefox_service, options = firefox_options)

In [4]:
df = pd.read_csv('Equity.csv', index_col = 0)
df.head()

,Security Code,Issuer Name,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument,Sector Name,Industry New Name,Igroup Name,ISubgroup Name,Y or N
500002,ABB India Limited,ABB,ABB India Limited,Active,A,2.0,INE117A01022,Heavy Electrical Equipment,Equity,Industrials,Heavy Electrical Equipment,Electrical Equipment,Heavy Electrical Equipment,NaN,Y
500003,Aegis Logistics Ltd.,AEGISLOG,AEGIS LOGISTICS LTD.,Active,A,1.0,INE208C01025,Trading - Gas,Equity,Energy,Trading - Gas,Gas,Trading - Gas,NaN,Y
500008,Amara Raja Batteries Ltd,AMARAJABAT,AMARA RAJA BATTERIES LTD.,Active,A,1.0,INE885A01032,Batteries - Automobile,Equity,Consumer Discretionary,Batteries - Automobile,Auto Components,Batteries - Automobile,NaN,Y
500009,"Ambalal Sarabhai Enterprise Ltd.,",AMBALALSA,AMBALAL SARABHAI ENTERPRISES LTD.,Active,X,10.0,INE432A01017,Pharmaceuticals,Equity,Healthcare,Pharmaceuticals,Pharmaceuticals & Biotechnology,Pharmaceuticals,NaN,Y
500010,Housing Development Finance Corp.Lt,HDFC,HOUSING DEVELOPMENT FINANCE CORP.LTD.,Active,A,2.0,INE001A01036,Housing Finance Company,Equity,Financial Services,Housing Finance Company,Finance,Housing Finance Company,NaN,Y


In [5]:
df.index

Int64Index([500002, 500003, 500008, 500009, 500010, 500012, 500013, 500014,
            500016, 500020,
            ...
            590104, 590106, 590107, 590108, 590109, 590110, 590115, 590122,
            590134, 780018],
           dtype='int64', length=3896)

### Loop to iterate over index and trigger javascript for downloading the file

Only possibility for error in this case was if link did not exist, hence try and except were used

time.sleep was used as it was required for the browser to wait for a second before loading the next link, otherwise file wouldn't get downloaded

Using this around 1,000 files were downloaded.

In [ ]:
#Loop to iterate over the index and trigger download on all links

for a in df.index:
    
    b = str(a)
    try:

        link = "http://financials.morningstar.com/ratios/r.html?t="+b+"&region=ind&culture=en-US"
        driver.get(link)
        
        js = 'exportKeyStat2CSV()'
        driver.execute_script(js)

    except:
        None

As a result of this loop, all files were downloaded in Downloads folder. They were moved to a different folder later on.

### Different Methods for rest of the files

for other files, url had a 10 digit code instead of the security code that started with 0p000.

To get this code, a query was made on google to return the first result of the search "<Security Code> key ratios morningstar"
    
From that point, it was easy to strip the url and extract the 10 digit url code and then use it in the same for loop as above

In [ ]:
#Identifying which files do not exist

from os.path import exists

for i in df.index:
    path = r'C:/Users/prash/Downloads/Python Stuff/Value Investing Project/Data/'+str(i)+" Key Ratios.csv"
    if not (os.path.exists(path)):
       df.loc[i,'Exists'] = 'N'
    
df[["Security Name", 'Exists']][df['Exists'] == 'N'].to_csv("Stocks without a file.csv")

In [ ]:
df2 = pd.read_csv('Stocks without a file.csv')

In [ ]:
from googlesearch import search

In [ ]:
for i in df.index:
    query = str(i)+" key ratios morningstar bse"
    for j in search(query, tld="co.in", num=2, stop=1, pause=2):
        None
    
    #One method for stripping the string -> use 4th and 5th occurence of '/'
    """z = [index for index, char in enumerate(j) if char == '/']
    start = z[3]+1
    end = z[4]-1"""
    
    
    #Second method for stripping the string -> identify consecutive 0s
    k = [index for index, char in enumerate(j) if char == '0']
    
    for x in range(len(k)):
        if ((len(k)-x-4) >=0):
            if (k[x]+1==k[x+1]) & (k[x]+2==k[x+2]) & (k[x]+3==k[x+3]):
                start = k[x]-2
                end = k[x] + 8
    
    
    df2.loc[i,'URL'] = j[start:end]
    print(str(i)+"  "+j[start:end])

and then using the same loop, around 2,000 more files were downloaded. This left us with around 500 files which contained indexes for ETFs as well. These were skipped

In [ ]:
#Loop to iterate over the index and trigger download on all links

for a in df.URL:
    
    b = str(a)
    try:

        link = "http://financials.morningstar.com/ratios/r.html?t="+b+"&region=ind&culture=en-US"
        driver.get(link)
        
        js = 'exportKeyStat2CSV()'
        driver.execute_script(js)

    except:
        None

## Data Cleaning + Summarizing

In [1]:
import pandas as pd
import os

### An Empty Dataframe was created with the following columns:

•	Code – Stock Code to be used as the Index

•	Name – Name of the stock

•	Industry – Industry of the company

•	Rev > 0 – Has the revenue been more than 0 in the past 10 years – 1 for Yes and 0 for No

•	NI > 0 – Has the net income been more than 0 in the past 10 years – 1 for Yes and 0 for No

•	D/E < 0.5 – Has Debt/Equity ratio been less than 0.5 in the past 10 years – 1 for Yes and 0 for No

•	GM Mean – Mean of Gross Margin Percentage

•	GM Std Dev – Standard Deviation of Gross Margin Percentage

•	GM CoV – Coefficient of Variation of Gross Margin Percentage

•	OM Mean – Mean of Operating Margin

•	OM Std Dev – Standard Deviation of Operating Margin

•	OM CoV – Coefficient of Variation of Operating Margin

•	FCF > 0 N.P. – Was Free Cashflow more than 0 before 2019, i.e., before Pandemic? – 1 for Yes and 0 for No

•	FCF > 0 Last 3 yrs – Has Free Cashflow been more than 0 in last 3 years – 1 for Yes and 0 for No

•	Shares Mil – Number of Shares in Million

•	Book Value per share

•	FCF Last 3 yrs – Mean of Free Cashflow for the last 3 years

•	Last FCF B.P. – Free Cashflow in the last year before pandemic

In [66]:
master_df = pd.read_csv('Equity.csv', index_col = 0)
master_df.head()

,Issuer Name,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument,Sector Name,Industry New Name,Igroup Name,ISubgroup Name
Security Code,,,,,,,,,,,,,
500002,ABB India Limited,ABB,ABB India Limited,Active,A,2.0,INE117A01022,Heavy Electrical Equipment,Equity,Industrials,Heavy Electrical Equipment,Electrical Equipment,Heavy Electrical Equipment
500003,Aegis Logistics Ltd.,AEGISLOG,AEGIS LOGISTICS LTD.,Active,A,1.0,INE208C01025,Trading - Gas,Equity,Energy,Trading - Gas,Gas,Trading - Gas
500008,Amara Raja Batteries Ltd,AMARAJABAT,AMARA RAJA BATTERIES LTD.,Active,A,1.0,INE885A01032,Batteries - Automobile,Equity,Consumer Discretionary,Batteries - Automobile,Auto Components,Batteries - Automobile
500009,"Ambalal Sarabhai Enterprise Ltd.,",AMBALALSA,AMBALAL SARABHAI ENTERPRISES LTD.,Active,X,10.0,INE432A01017,Pharmaceuticals,Equity,Healthcare,Pharmaceuticals,Pharmaceuticals & Biotechnology,Pharmaceuticals
500010,Housing Development Finance Corp.Lt,HDFC,HOUSING DEVELOPMENT FINANCE CORP.LTD.,Active,A,2.0,INE001A01036,Housing Finance Company,Equity,Financial Services,Housing Finance Company,Finance,Housing Finance Company


In [67]:
df = pd.DataFrame(columns = ["Code", "Name", "Industry", "Rev > 0", "NI > 0", "D/E < 0.5", "GM Mean", "GM Std Dev", "GM CoV", "OM Mean", "OM Std Dev", "OM CoV", "FCF > 0 N.P.", "FCF > 0 Last 3 yrs", "Shares Mil", "Book Value per share", "FCF Last 3 yrs", "Last FCF B.P."], index = list(range(len(master_df))))
df.head()

,Code,Name,Industry,Rev > 0,NI > 0,D/E < 0.5,GM Mean,GM Std Dev,GM CoV,OM Mean,OM Std Dev,OM CoV,FCF > 0 N.P.,FCF > 0 Last 3 yrs,Shares Mil,Book Value per share,FCF Last 3 yrs,Last FCF B.P.
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
df.Code = master_df.index
df.set_index('Code', inplace = True)

In [69]:
df.head(2)

,Name,Industry,Rev > 0,NI > 0,D/E < 0.5,GM Mean,GM Std Dev,GM CoV,OM Mean,OM Std Dev,OM CoV,FCF > 0 N.P.,FCF > 0 Last 3 yrs,Shares Mil,Book Value per share,FCF Last 3 yrs,Last FCF B.P.
Code,,,,,,,,,,,,,,,,,
500002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
#df.drop('Code', axis = 1, inplace = True)

In [70]:
df['Name'] = master_df['Security Name']
df['Industry'] = master_df['Industry']

In [71]:
df.head()

,Name,Industry,Rev > 0,NI > 0,D/E < 0.5,GM Mean,GM Std Dev,GM CoV,OM Mean,OM Std Dev,OM CoV,FCF > 0 N.P.,FCF > 0 Last 3 yrs,Shares Mil,Book Value per share,FCF Last 3 yrs,Last FCF B.P.
Code,,,,,,,,,,,,,,,,,
500002,ABB India Limited,Heavy Electrical Equipment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500003,AEGIS LOGISTICS LTD.,Trading - Gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500008,AMARA RAJA BATTERIES LTD.,Batteries - Automobile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500009,AMBALAL SARABHAI ENTERPRISES LTD.,Pharmaceuticals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500010,HOUSING DEVELOPMENT FINANCE CORP.LTD.,Housing Finance Company,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
#To save up memory
del master_df

In [73]:
#Removing indexes for which files don't exist

from os.path import exists

for i in df.index:
    path = r'C:/Users/prash/Downloads/Python Stuff/Value Investing Project/Data/'+str(i)+" Key Ratios.csv"
    if not (os.path.exists(path)):
       df.drop(i, inplace = True) 

In [74]:
for i in df.index:
        
    path = r'C:/Users/prash/Downloads/Python Stuff/Value Investing Project/Data/'+str(i)+" Key Ratios.csv"
    dataframe = pd.read_csv(path, header = None, names = range(12))
    
    
    #There are some dataframes for which files were downloaded, however, there's no data. To rectify that error, this command has been used.
    if dataframe.iloc[0,0] == 'We’re sorry. There is no available information in our database to display.':
        continue
    
    #This is to read the key ratios csv file. It contains multiple tables in one sheet thus this command was used
    table_names = ["Financials", "Key Ratios -> Profitability", "Profitability", "Key Ratios -> Growth", "Key Ratios -> Cash Flow", "Key Ratios -> Financial Health", "Liquidity/Financial Health", "Key Ratios -> Efficiency Ratios"]
    groups = dataframe[0].isin(table_names).cumsum()
    
    #This creates a dictionary file with table name as the key and table as the value
    tables = {g.iloc[0,0]: g.iloc[1:] for k,g in dataframe.groupby(groups)}
    
    
    #The first table is just plain text without any value thus we remove it
    del tables[list(tables.keys())[0]]
    
    #Merging these two tables as they're primarily the same table
    tables['Liquidity/Financial Health']=pd.concat([dataframe[dataframe[0]=='Liquidity/Financial Health'],tables['Liquidity/Financial Health']], axis = 0)
    
    #Function for removing commas from numeric values and returning them as float values
    def str_to_number(x):
        if type(x) == str:
            if x.startswith('-'):
                a = '-'+"".join(x[1:].split(','))
                return float(a)
            else:
                a = "".join(x.split(','))
                return float(a)
        else:
            return float(x)
    
    #Function for cleaning tables extracted from key ratios.csv
    def clean(x):
        x = x.reset_index().drop(['index'], axis = 1)
        x.columns = x.iloc[0]
        x = x[1:]
        x = x.set_index(x.columns.values[0])
        return x
    
    #Function for cleaning the transposed Index. Some dataframes had the values 2019-03 whereas some had 2019-12.
    def index_clean (x):
        if x.index[-1] == 'TTM':
            x = x.set_index(pd.DatetimeIndex(x.index[:-1]).year.astype(str).append(x.index[-1:]))
        elif x.index[-2] == 'TTM':
            x = x.set_index(pd.DatetimeIndex(x.index[:-2]).year.astype(str).append(x.index[-2:]))
            x = x.iloc[:-1, :]
        return x

    #Loop to clean all tables
    for j in tables.keys():

        if j in ["Financials", "Growth"]:
            tables[j].iloc[0,0] = 'Heads'
        tables[j] = clean(tables[j])
    
    #Extracting final tables
    kr_financials = tables['Financials'].transpose()
    kr_liquidity = tables['Liquidity/Financial Health'].transpose()
    kr_financials['Gross Margin %'] = kr_financials['Gross Margin %'].astype(float)
    kr_financials['Operating Margin %'] = kr_financials['Operating Margin %'].astype(float)
    
    #A few stocks had values in USD Mil instead of INR, this script converts these values into INR
    #Other columns weren't converted into INR as they weren't required in our calculations
    
    if 'Revenue USD Mil' in kr_financials.columns:
        kr_financials['Revenue USD Mil'] = kr_financials['Revenue USD Mil'].apply(str_to_number) * 77.49
        #kr_financials['Operating Income USD Mil'] = kr_financials['Operating Income USD Mil'] * 77.49
        kr_financials['Net Income USD Mil'] = kr_financials['Net Income USD Mil'].apply(str_to_number) * 77.49
        kr_financials['Earnings Per Share USD'] = kr_financials['Earnings Per Share USD'].apply(str_to_number) * 77.49
        #kr_financials['Dividends USD'] = kr_financials['Dividends USD'] * 77.49
        kr_financials['Book Value Per Share * USD'] = kr_financials['Book Value Per Share * USD'].apply(str_to_number) * 77.49
        #kr_financials['Operating Cash Flow USD Mil'] = kr_financials['Operating Cash Flow USD Mil'] * 77.49
        #kr_financials['Cap Spending USD Mil'] = kr_financials['Cap Spending USD Mil'] * 77.49
        kr_financials['Free Cash Flow USD Mil'] = kr_financials['Free Cash Flow USD Mil'].apply(str_to_number) * 77.49
        kr_financials.columns = ['Revenue INR Mil', 'Gross Margin %', 'Operating Income USD Mil',
       'Operating Margin %', 'Net Income INR Mil', 'Earnings Per Share INR',
       'Dividends USD', 'Payout Ratio % *', 'Shares Mil',
       'Book Value Per Share * INR', 'Operating Cash Flow USD Mil',
       'Cap Spending USD Mil', 'Free Cash Flow INR Mil',
       'Free Cash Flow Per Share * USD', 'Working Capital USD Mil']
    
    
    #Further cleaning - ensuring that we've got the correct data types
    kr_financials['Revenue INR Mil'] = kr_financials['Revenue INR Mil'].apply(str_to_number) 
    kr_financials['Net Income INR Mil'] = kr_financials['Net Income INR Mil'].apply(str_to_number)
    kr_liquidity['Debt/Equity'] = kr_liquidity['Debt/Equity'].astype(float)
    kr_financials['Free Cash Flow INR Mil'] = kr_financials['Free Cash Flow INR Mil'].apply(str_to_number)
    kr_financials['Free Cash Flow INR Mil'] = kr_financials['Free Cash Flow INR Mil'].apply(str_to_number)  
    kr_financials = index_clean(kr_financials)
    kr_liquidity = index_clean(kr_liquidity)
    kr_financials['Shares Mil'] = kr_financials['Shares Mil'].apply(str_to_number)
    
    #Freeing up memory
    del dataframe
    del tables
    del table_names
    del groups
    
    
    #Now we start filling our main dataframe
    
    if (min(kr_financials['Revenue INR Mil'])>0):
        df.loc[i,"Rev > 0"] = 1
    else:
        df.loc[i,"Rev > 0"] = 0
        
    if (min(kr_financials["Net Income INR Mil"])>0):
        df.loc[i,"NI > 0"] = 1
    else:
        df.loc[i,"NI > 0"] = 0
    
    if(max(kr_liquidity['Debt/Equity'])<0.5):
        df.loc[i, "D/E < 0.5"] = 1
    else:
        df.loc[i, "D/E < 0.5"] = 0
   
    df.loc[i, "GM Mean"] = kr_financials["Gross Margin %"].mean()
    df.loc[i, "GM Std Dev"] = kr_financials["Gross Margin %"].std()
    
    df.loc[i, "OM Mean"] = kr_financials["Operating Margin %"].mean()
    df.loc[i, "OM Std Dev"] = kr_financials["Operating Margin %"].std()
    
    
    
    #FCF > 0 N.P.
    
    index_lessthan2019 = kr_financials.index[:-1].astype(int)[kr_financials.index[:-1].astype(int)<=2019]
    index_lessthan2019 = index_lessthan2019.astype(str)
    index_morethan2019 = kr_financials.index[:-1].astype(int)[kr_financials.index[:-1].astype(int)>2019]
    index_morethan2019 = index_morethan2019.astype(str)
    
    if(float(kr_financials.loc[index_lessthan2019,['Free Cash Flow INR Mil']].min()) > 0):
        df.loc[i, "FCF > 0 N.P."] = 1
    else:
        df.loc[i, "FCF > 0 N.P."] = 0

    #FCF Last 3 years. Try command has been used as some stocks didn't have complete data for these years
    try:     
        if(float(kr_financials.loc[index_morethan2019,['Free Cash Flow INR Mil']].min()) > 0):
            df.loc[i, "FCF > 0 Last 3 yrs"] = 1
        else:
            df.loc[i, "FCF > 0 Last 3 yrs"] = 0
    except:
        None
    
    df.loc[i, "Shares Mil"] = kr_financials.loc["TTM","Shares Mil"]
    df.loc[i, "Book Value per share"] = kr_financials.loc["TTM","Book Value Per Share * INR"]
  
    #FCF Last 3 years Mean. Try command has been used as some stocks didn't have complete data for these years
    try:
        df.loc[i, "FCF Last 3 yrs"] = float(kr_financials.loc[index_morethan2019,['Free Cash Flow INR Mil']].mean())
    except:
       None
    
    #Free Cashflow in the last year before pandemic
    df.loc[i, "Last FCF B.P."] = kr_financials.loc[str(index_lessthan2019.astype(int).max()), "Free Cash Flow INR Mil"]

In [75]:
df.to_csv('Dataframe Screened.csv')

# Filtering + Further Extraction of Data

In [ ]:
import pandas as pd

In [76]:
df = pd.read_csv('Dataframe Screened.csv', index_col = 0)
df.head()

,Name,Industry,Rev > 0,NI > 0,D/E < 0.5,GM Mean,GM Std Dev,GM CoV,OM Mean,OM Std Dev,OM CoV,FCF > 0 N.P.,FCF > 0 Last 3 yrs,Shares Mil,Book Value per share,FCF Last 3 yrs,Last FCF B.P.
Code,,,,,,,,,,,,,,,,,
500002,ABB India Limited,Heavy Electrical Equipment,1.0,1.0,0.0,28.845455,1.929437,NaN,6.972727,1.449200,NaN,0.0,1.0,212.0,190.89,3369.000000,5295.0
500003,AEGIS LOGISTICS LTD.,Trading - Gas,1.0,1.0,1.0,9.872727,5.150552,NaN,5.127273,3.186563,NaN,0.0,0.0,351.0,60.49,291.500000,4022.0
500008,AMARA RAJA BATTERIES LTD.,Batteries - Automobile,1.0,1.0,1.0,30.236364,1.714219,NaN,12.372727,1.878346,NaN,0.0,1.0,171.0,256.11,3765.000000,127.0
500009,AMBALAL SARABHAI ENTERPRISES LTD.,Pharmaceuticals,1.0,0.0,1.0,35.181818,20.463666,NaN,-6.481818,11.450486,NaN,0.0,0.0,77.0,14.31,-77.000000,84.0
500010,HOUSING DEVELOPMENT FINANCE CORP.LTD.,Housing Finance Company,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1827.0,992.00,-336126.333333,-445132.0


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3528 entries, 500002 to 780018
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  3528 non-null   object 
 1   Industry              3528 non-null   object 
 2   Rev > 0               3473 non-null   float64
 3   NI > 0                3473 non-null   float64
 4   D/E < 0.5             3473 non-null   float64
 5   GM Mean               3332 non-null   float64
 6   GM Std Dev            3309 non-null   float64
 7   GM CoV                0 non-null      float64
 8   OM Mean               3357 non-null   float64
 9   OM Std Dev            3331 non-null   float64
 10  OM CoV                0 non-null      float64
 11  FCF > 0 N.P.          3473 non-null   float64
 12  FCF > 0 Last 3 yrs    3473 non-null   float64
 13  Shares Mil            3460 non-null   float64
 14  Book Value per share  3424 non-null   float64
 15  FCF Last 3 yrs

In [78]:
#Stocks for which GM column wasn't filled
df = df[~df['GM Std Dev'].isna()]

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3309 entries, 500002 to 780018
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  3309 non-null   object 
 1   Industry              3309 non-null   object 
 2   Rev > 0               3309 non-null   float64
 3   NI > 0                3309 non-null   float64
 4   D/E < 0.5             3309 non-null   float64
 5   GM Mean               3309 non-null   float64
 6   GM Std Dev            3309 non-null   float64
 7   GM CoV                0 non-null      float64
 8   OM Mean               3307 non-null   float64
 9   OM Std Dev            3306 non-null   float64
 10  OM CoV                0 non-null      float64
 11  FCF > 0 N.P.          3309 non-null   float64
 12  FCF > 0 Last 3 yrs    3309 non-null   float64
 13  Shares Mil            3297 non-null   float64
 14  Book Value per share  3268 non-null   float64
 15  FCF Last 3 yrs

In [80]:
#Stock for which OM column wasn't filled
df = df[~df['OM Std Dev'].isna()]

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3306 entries, 500002 to 780018
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  3306 non-null   object 
 1   Industry              3306 non-null   object 
 2   Rev > 0               3306 non-null   float64
 3   NI > 0                3306 non-null   float64
 4   D/E < 0.5             3306 non-null   float64
 5   GM Mean               3306 non-null   float64
 6   GM Std Dev            3306 non-null   float64
 7   GM CoV                0 non-null      float64
 8   OM Mean               3306 non-null   float64
 9   OM Std Dev            3306 non-null   float64
 10  OM CoV                0 non-null      float64
 11  FCF > 0 N.P.          3306 non-null   float64
 12  FCF > 0 Last 3 yrs    3306 non-null   float64
 13  Shares Mil            3294 non-null   float64
 14  Book Value per share  3265 non-null   float64
 15  FCF Last 3 yrs

In [82]:
#Saved for future analysis as D/E for finance stocks won't work
financial_services = df[(df.Industry == "Financial Services")& (df['Rev > 0'] == 1) & (df['NI > 0'] == 1) ].copy()

In [83]:
financial_services.to_csv('financial services.csv')

In [84]:
#Now our dataframe consists of debt free stocks that haven't made losses in past 10 years

df = df[(df['Rev > 0'] == 1) & (df['NI > 0'] == 1) & (df['D/E < 0.5'] == 1)]
df = df[~(df.Industry == 'Financial Services')]

df['GM CoV'] = df['GM Std Dev']/df['GM Mean']
df['OM CoV'] = df['OM Std Dev']/df['OM Mean']

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 301 entries, 500003 to 590006
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  301 non-null    object 
 1   Industry              301 non-null    object 
 2   Rev > 0               301 non-null    float64
 3   NI > 0                301 non-null    float64
 4   D/E < 0.5             301 non-null    float64
 5   GM Mean               301 non-null    float64
 6   GM Std Dev            301 non-null    float64
 7   GM CoV                301 non-null    float64
 8   OM Mean               301 non-null    float64
 9   OM Std Dev            301 non-null    float64
 10  OM CoV                301 non-null    float64
 11  FCF > 0 N.P.          301 non-null    float64
 12  FCF > 0 Last 3 yrs    301 non-null    float64
 13  Shares Mil            301 non-null    float64
 14  Book Value per share  301 non-null    float64
 15  FCF Last 3 yrs 

In [86]:
df.to_csv('Profitable and debt free stocks.csv')

In [87]:
df.head()

,Name,Industry,Rev > 0,NI > 0,D/E < 0.5,GM Mean,GM Std Dev,GM CoV,OM Mean,OM Std Dev,OM CoV,FCF > 0 N.P.,FCF > 0 Last 3 yrs,Shares Mil,Book Value per share,FCF Last 3 yrs,Last FCF B.P.
Code,,,,,,,,,,,,,,,,,
500003,AEGIS LOGISTICS LTD.,Trading - Gas,1.0,1.0,1.0,9.872727,5.150552,0.521695,5.127273,3.186563,0.621493,0.0,0.0,351.0,60.49,291.500000,4022.0
500008,AMARA RAJA BATTERIES LTD.,Batteries - Automobile,1.0,1.0,1.0,30.236364,1.714219,0.056694,12.372727,1.878346,0.151813,0.0,1.0,171.0,256.11,3765.000000,127.0
500027,ATUL LTD.,Specialty Chemicals,1.0,1.0,1.0,44.545455,4.189359,0.094047,15.063636,3.071571,0.203906,1.0,0.0,30.0,1500.64,1810.666667,1952.0
500039,BANCO PRODUCTS (INDIA) LTD.-$,Auto Components & Equipments,1.0,1.0,1.0,44.154545,3.963928,0.089774,9.427273,1.854773,0.196745,1.0,1.0,72.0,128.44,711.000000,208.0
500052,BHANSALI ENGINEERING POLYMERS LTD.-$,Specialty Chemicals,1.0,1.0,1.0,27.018182,13.054411,0.483171,13.872727,12.864143,0.927297,0.0,1.0,166.0,59.15,1478.000000,169.0


In [88]:
#For Consistency in Gross Margin and Operating Margin

df = df[(df['GM CoV'] <0.3) & (df['OM CoV'] <0.3)]

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138 entries, 500008 to 590006
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  138 non-null    object 
 1   Industry              138 non-null    object 
 2   Rev > 0               138 non-null    float64
 3   NI > 0                138 non-null    float64
 4   D/E < 0.5             138 non-null    float64
 5   GM Mean               138 non-null    float64
 6   GM Std Dev            138 non-null    float64
 7   GM CoV                138 non-null    float64
 8   OM Mean               138 non-null    float64
 9   OM Std Dev            138 non-null    float64
 10  OM CoV                138 non-null    float64
 11  FCF > 0 N.P.          138 non-null    float64
 12  FCF > 0 Last 3 yrs    138 non-null    float64
 13  Shares Mil            138 non-null    float64
 14  Book Value per share  138 non-null    float64
 15  FCF Last 3 yrs 

In [90]:
df.to_csv('Consistently profitable and debt free stocks.csv')

In [91]:
df['FCF Strength'] = (df['FCF > 0 N.P.'] + df['FCF > 0 Last 3 yrs'])/2

In [92]:
df = df[df['FCF Strength'] == 1]

#Stronger stocks with positive cashflow all around

In [93]:
df['Valued Price'] = (df['FCF Last 3 yrs']/0.02)/df['Shares Mil']
df['Buy Price'] = df['Valued Price']/2
df['Sell Price'] = df['Valued Price']*2

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 500039 to 540596
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  29 non-null     object 
 1   Industry              29 non-null     object 
 2   Rev > 0               29 non-null     float64
 3   NI > 0                29 non-null     float64
 4   D/E < 0.5             29 non-null     float64
 5   GM Mean               29 non-null     float64
 6   GM Std Dev            29 non-null     float64
 7   GM CoV                29 non-null     float64
 8   OM Mean               29 non-null     float64
 9   OM Std Dev            29 non-null     float64
 10  OM CoV                29 non-null     float64
 11  FCF > 0 N.P.          29 non-null     float64
 12  FCF > 0 Last 3 yrs    29 non-null     float64
 13  Shares Mil            29 non-null     float64
 14  Book Value per share  29 non-null     float64
 15  FCF Last 3 yrs  

In [95]:
df[["Name", "Industry", "FCF Strength", "Valued Price", "Buy Price", "Sell Price"]].to_csv('Stocks with Prices.csv')

In [96]:
df[["Name", "Industry", "FCF Strength", "Valued Price", "Buy Price", "Sell Price"]]

,Name,Industry,FCF Strength,Valued Price,Buy Price,Sell Price
Code,,,,,,
500039,BANCO PRODUCTS (INDIA) LTD.-$,Auto Components & Equipments,1.0,493.750000,246.875000,987.500000
500096,DABUR INDIA LTD.,Personal Care,1.0,416.224854,208.112427,832.449709
500144,FINOLEX CABLES LTD.,Cables - Electricals,1.0,467.647059,233.823529,935.294118
500150,FOSECO INDIA LTD.,Specialty Chemicals,1.0,1675.000000,837.500000,3350.000000
500331,PIDILITE INDUSTRIES LTD.,Specialty Chemicals,1.0,909.037328,454.518664,1818.074656
500820,ASIAN PAINTS LTD.,Paints,1.0,1463.190824,731.595412,2926.381648
500825,BRITANNIA INDUSTRIES LTD.,Packaged Foods,1.0,2488.727524,1244.363762,4977.455048
500875,ITC LTD.,Diversified FMCG,1.0,465.558532,232.779266,931.117064
505714,GABRIEL INDIA LTD.-$,Auto Components & Equipments,1.0,314.930556,157.465278,629.861111
